In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os 
os.chdir('/content/drive/Shareddrives/AC297r Project - RFCx /code')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import json
import numpy as np

import tensorflow as tf
from tensorflow.keras import backend as K
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import ModelCheckpoint, LambdaCallback

from datagen import get_files_and_labels, scalespec, preprocess, DataGenerator
from learningrate import warmup_cosine_decay, WarmUpCosineDecayScheduler

import time


In [ ]:
model_out = './model' # path to output model
data_dir = '/content/drive/Shareddrives/AC297r Project - RFCx /image_data/puerto-rico/train/audio' # path to directory with spectrogram data
# expected format:
#     data_dir/
#         p/
#             <class_1>/
#                 <audio_filename>.npy
#                 ...
#             <class_2>/
#                 <audio_filename>.npy
#                 ...
#         n/
#             <cl ass_1>/
#                 <audio_filename>.npy
#                 ...
#             <class_2>/
#                 <audio_filename>.npy
#                 ...


In [ ]:
# specify list of target classes
class_list = os.listdir(data_dir+'/p/')
num_classes = len(class_list)
num_classes


45

In [ ]:
%%time
train_split = 0.8

# generate positive train file paths
files_train_p, files_val_p, labels = get_files_and_labels(data_dir+'/p/',
                                                          train_split=train_split,
                                                          random_state=42,
                                                          classes=class_list)

# generate negative train file paths
files_train_n, files_val_n, labels_n = get_files_and_labels(data_dir+'/n/',
                                                            train_split=train_split,
                                                            random_state=42,
                                                            classes=class_list) 

labels_rev = dict((v,k) for (k,v) in labels.items())
files_train_n = [i for i in files_train_n if i.split('/')[-2] in list(labels.keys())]
files_train = files_train_p + files_train_n
files_val = files_val_p + files_val_n


CPU times: user 163 ms, sys: 34.3 ms, total: 198 ms
Wall time: 1.06 s


In [ ]:
# %%time 

# path = "/content/drive/Shareddrives/AC297r Project - RFCx /image_data_train_val_split/"

# if not os.path.exists(path):
#    os.mkdir(path)
#    os.mkdir(path + "/p/") # positive samples directory 
#    os.mkdir(path + "/n/") # negative samples directory

# # i = 0
# # file = files_train_p[0]

# # Save positive samples 
# for i, file in enumerate(files_train_p):

#   if i % 1000 == 0:
#     print("train positive: ", i, " / ", len(files_train_p))

#   class_name = file.split('/')[-2]
#   if not os.path.exists(path + "/p/" + class_name):
#     os.mkdir(path + "/p/" + class_name)

#   # Copy the data file to new path 
#   sample_data = np.load(file)
#   file_new = path + "/p/" + class_name + "/" + file.split('/')[-1]
#   np.save(file_new, sample_data)



In [ ]:
%%time 

# Create directory 
path = "/content/drive/Shareddrives/AC297r Project - RFCx /image_data_train_val_split/"

if not os.path.exists(path):
  os.mkdir(path)
  os.mkdir(path + "/train/")
  os.mkdir(path + "/val/")

path_train_p = path + "/train/p/"
path_train_n = path + "/train/n/"
path_val_p = path + "/val/p/"
path_val_n = path + "/val/n/"

path_all = [path_train_p, path_train_n, path_val_p, path_val_n]

for p in path_all:
  if not os.path.exists(p):
   os.mkdir(p)  



def transfer_data(files_input, path_output):
  """
  files_input: a list of all input data files 
  path_output: directory for saving output data files 
  
  Example - transfer_data(files_train_p, path_train_p)
  """

  start = time.time()

  for i, file in enumerate(files_input):

    if i % 100 == 0:
      print(i, " / ", len(files_input))
      end = time.time()
      print("time elapsed: ", round(end - start, 2), "s")
      time.sleep(1)
      start = end

    class_name = file.split('/')[-2]
    if not os.path.exists(path_output + class_name):
      os.mkdir(path_output + class_name)

    # Copy the data file to new path 
    sample_data = np.load(file, allow_pickle=True)
    file_output_path = path_output + class_name + "/" + file.split('/')[-1]
    np.save(file_output_path, sample_data)



print("train positive: ")
transfer_data(files_train_p, path_train_p)

# print("train negative: ")
# transfer_data(files_train_n, path_train_n)

# print("val postive: ")
# transfer_data(files_val_p, path_val_p)

# print("val negative: ")
# transfer_data(files_val_n, path_val_n)





train positive: 
0  /  6445
time elapsed:  0.0 s
100  /  6445
time elapsed:  2.13 s
200  /  6445
time elapsed:  2.04 s
300  /  6445
time elapsed:  2.04 s
400  /  6445
time elapsed:  2.24 s
500  /  6445
time elapsed:  2.03 s
600  /  6445
time elapsed:  2.35 s
700  /  6445
time elapsed:  2.04 s
800  /  6445
time elapsed:  2.47 s
900  /  6445
time elapsed:  2.74 s
1000  /  6445
time elapsed:  3.81 s
1100  /  6445
time elapsed:  2.14 s
1200  /  6445
time elapsed:  2.18 s
1300  /  6445
time elapsed:  2.36 s
1400  /  6445
time elapsed:  18.15 s
1500  /  6445
time elapsed:  41.49 s
1600  /  6445
time elapsed:  41.26 s
1700  /  6445
time elapsed:  41.2 s
1800  /  6445
time elapsed:  40.67 s
1900  /  6445
time elapsed:  41.05 s
2000  /  6445
time elapsed:  40.81 s


KeyboardInterrupt: ignored

In [ ]:
print("train positive: ")
transfer_data(files_train_p[3000 : 5000], path_train_p)

train positive: 
0  /  2000
time elapsed:  0.0 s
100  /  2000
time elapsed:  39.86 s
200  /  2000
time elapsed:  40.14 s
300  /  2000
time elapsed:  40.37 s
400  /  2000
time elapsed:  40.53 s
500  /  2000
time elapsed:  39.7 s
600  /  2000
time elapsed:  41.19 s
700  /  2000
time elapsed:  41.13 s
800  /  2000
time elapsed:  39.67 s
900  /  2000
time elapsed:  41.8 s
1000  /  2000
time elapsed:  40.55 s
1100  /  2000
time elapsed:  41.11 s
1200  /  2000
time elapsed:  40.69 s
1300  /  2000
time elapsed:  40.31 s
1400  /  2000
time elapsed:  39.37 s
1500  /  2000
time elapsed:  40.48 s
1600  /  2000
time elapsed:  39.49 s
1700  /  2000
time elapsed:  39.49 s
1800  /  2000
time elapsed:  39.87 s
1900  /  2000
time elapsed:  41.3 s
